# Data reduction workflow

This notebook includes a full data reduction workflow for SHIVA experimental data. However, some part were done with external resources.

- Data reduction of raw data into .mat files was done with a matlab UI: [SHIVAunix](https://github.com/aretu/RSF-directmodels).
- Estimation of permeability was done with a series of matlab functions authored by Julian Mecklenburgh.
- Modelling was donw with a series of matlab functions: [RSF-directmodels](https://github.com/aretu/RSF-directmodels)

### Import section

In [1]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import scipy.signal as ssi
import scipy.io as sio
import pandas as pd
import os

%matplotlib qt
plt.rcParams.update({'font.size': 20})

### Defining paths

In [33]:
#paths

plots_path='../plots/'
out_perm_data_path='../data/experiments/SHIVA_perm_cycles/'
table_path='../data/experiments/tables/'
shiva_red_unix_path='../data/experiments/shiva_red_shivaUNIX/'
shiva_red_jupyter_path='../data/experiments/shiva_red_jupyter/'

inpath=shiva_red_unix_path
if os.path.exists(inpath):
    print(inpath+' exists')

../data/experiments/shiva_red_shivaUNIX/ exists


### Importing events from table
Events were defined visually or with script below, then the table was edited in excel.

In [3]:
# import events table

dt={'event_id':'str','event_name':'str','s1949':'int','s1984_r1':'int','s1984_r2':'int','brava':'int','s1994':'int','s1995':'int','s1996':'int','s1997':'int'}

indexes=pd.read_excel(table_path+'experiment_stages.xlsx',sheet_name='indexes',usecols=[0,1,2,3,4,5,6,7,8,9],dtype=dt)
indexes.head()

,exp,event_id,event_name,start,end
0,s1994,1,before preshear,9394,9651
1,s1994,2,after preshear,82710,83890
2,s1994,3,after preshear with shear stress,109400,162200
3,s1995,1,before preshear,9451,9750
4,s1995,2,after preshear,41980,44560


### Basic functionality to:
- Transform .mat files to dataframes
- Map variable names and calculate variables
- Slice data based on events names
- Save figures

In [4]:
def mat2df(mat):
    dfs = {}

    # Loop through each key in the dictionary and create a DataFrame for each array
    for key in mat:
        if isinstance(mat[key], np.ndarray):
            dfs[key] = pd.DataFrame(mat[key],columns=[key])

    # Combine all the DataFrames in the dictionary into a single DataFrame
    df = pd.concat(dfs.values(), axis=1)
    return df

def mapper(exp):
    exp_o=[]
    
    t0=indexes.loc[indexes['event_name']=='start preshear',expname].iloc[0]

    if ismat==True:
        exp_o=exp
        timezero=exp_o['Time'][t0]

        exp_o['Time']=(exp_o['Time']-timezero)/1000
        exp_o['Time_hour']=exp_o['Time']/60/60
        exp_o['Time_min']=exp_o['Time']/60

        if material=='rock' and machine=='shiva':
            exp_o['effstress']=exp_o['Normal']-exp_o['Pf']*1.5625 #Pier's correction
        else:
            exp_o['effstress']=exp_o['Normal']-exp_o['Pf']

        exp_o['InternalPressure']=exp_o['PumpPressure'] #when cosino is dead

        exp_o['Mu_eff']=exp_o['shear1']/exp_o['effstress'] 
        exp_o['Mu_app']=exp_o['shear1']/exp_o['Normal']    


    else:
        if machine=='brava':
            print('brava')
            # dsv names
            dict={'Time (s)':'Time',
                    'Shear Stress (MPa)':'shear1',
                    'Normal Stress (MPa)':'Normal',
                    'Pore Pressure A (MPa)':'PumpPressure', # we have two upstream Pf in SHIVA: problem!
                    'Pore Pressure B (MPa)':'Pf',
                    'Confining Pressure (MPa)':'Pc',
                    'H. Displacement (mm)':'thickness_high',
                    'Sip Velocity (µm/S)':'vel',
                    'V. Displacement (mm)':'slip'
                    }
            
            exp_o=exp.rename(columns=dict)
            
            timezero=exp_o['Time'][t0]

            exp_o['effstress']=exp_o['Normal']-exp_o['Pf']

            exp_o['vel']=exp_o['vel']/1000000
            exp_o['slip']=exp_o['slip']/1000
            exp_o['Slip_Enc_1']=exp_o['slip']
            exp_o['Mu_eff']=exp_o['shear1']/(exp_o['effstress'])   
            exp_o['Mu_app']=exp_o['shear1']/exp_o['Normal']    

            exp_o['Time']=(exp_o['Time']-timezero) 
            exp_o['Time_min']=exp_o['Time']/60
            exp_o['Time_hour']=exp_o['Time']/60/60


        elif machine=='shiva':
            print('shiva')

        # dsv names

            dict={'Time (s)':'Time',
                    'ShearStress (MPa)':'shear1',
                    'NormalStress (MPa)':'Normal',
                    'IscoPumpBPressure (MPa)':'InternalPressure',
                    'BigMotorShearStress (MPa)':'BigMotorShearStress',
                    'GemsPressure (MPa)':'Pf',
                    'IscoPumpAPressure (MPa)':'Pc',
                    'Thickness (mm)':'thickness_high',
                    'Shortening (mm)':'LVDT_high',
                    'Velocity (m/s)':'vel',
                    'Slip (m)':'slip',
                    'Enc1Slip (m)':'Slip_Enc_1',
                    'Enc2Slip (m)':'Slip_Enc_2',
                    # 'vel':'SlipVel_Enc_1',
                    'EffectiveNormalStress (MPa)':'effstress'
                    }

            # exp.rename(columns=dict,inplace=True)
            exp_o=exp.rename(columns=dict)
            timezero=exp_o['Time'][t0]
            
            exp_o['Mu_eff']=exp_o['shear1']/exp_o['effstress']    
            exp_o['Mu_app']=exp_o['shear1']/exp_o['Normal']    

            exp_o['Time']=(exp_o['Time']-timezero) 
            exp_o['Time_hour']=exp_o['Time']/60/60
            exp_o['Time_min']=exp_o['Time']/60
   
    return exp_o

def slicer(exp,start,end):
    
    a=indexes.loc[indexes['event_name']==start,expname].iloc[0].copy()
    b=indexes.loc[indexes['event_name']==end,expname].iloc[0].copy()

    #slice
    slexp_o = exp.iloc[a:b]


    c=indexes[indexes['event_name']==start].index.values
    d=indexes[indexes['event_name']==end].index.values
  
    slindexes_o=indexes[c[0]:d[0]]

    return slexp_o,slindexes_o


def savefigures(name,path,figure):
    figure.savefig(path+str(name))
    figure.savefig(path+str(name)+'.svg')

### Figures creation

In [5]:
## All figure are here

def figure_mohr(exp,indexes,expname,plots_path):
    # %% figure 1 with resume of stages and events
    slexp,slindexes=slicer(exp,'start preshear','end fast slip')

    mohr,ax=plt.subplots(figsize=(14,9))

    df=slexp
    idxs=indexes

    ax.plot(df['effstress'],df['shear1'],'ok')

    ax.set_title(expname)
    ax.set_ylabel('Shear Stress [MPa]')
    ax.set_xlabel('Effective Normal Stress [MPa]')
    ax.set_ylim(-0.05,5.05)
    ax.set_xlim(-0.05,10.05)
    
    ax.plot([0,10],[0,5.3],'--k')
    ax.plot([0,10],[0,4.9],'--k')

    savefigures(expname+'_mohr',plots_path,mohr)
    # plt.close(mohr)  

def figure1(exp,indexes,expname,plots_path):
    # %% figure 1 with resume of stages and events
    slexp,slindexes=slicer(exp,'start preshear','end last permeability')

    fig1,ax=plt.subplots(figsize=(14,9))

    df=slexp
    idxs=indexes

    ax.plot(df['Time_hour'],df['Mu_eff'],'-k')

    ax.set_title(expname)
    ax.set_ylabel('Effective friction')
    ax.set_xlabel('Time [hours]')
    ax.set_ylim(-0.05,0.8)
    # ax.set_xlim(0,df['Time_hour'][-1])
    
    #pump or internalpressure?
    ax1=ax.twinx()
    ax1.plot(df['Time_hour'],df['PumpPressure'],'-c')
    ax1.plot(df['Time_hour'],df['Pf'],'-b')
    ax1.set_ylabel('Pore pressure [MPa]')
    ax1.set_ylim(0,7.5)

    for i in slindexes[expname]:
        try:
            # print(i)
            # print(indexes.loc[indexes[expname]==i,'event_name'].iloc[0])
            ax.axvline(x = df['Time_hour'][i], color = 'k', label = idxs.loc[idxs[expname]==i,'event_name'].iloc[0],ymax=1)
        except:
            print('index out of bound')

    savefigures(expname+'_fig1',plots_path,fig1)
    # plt.close(fig1)    

def figure2(exp,indexes,expname,plots_path):
    # %% figure 2 normal stress vs shear stress
    # % only injection section

    slexp,slindexes=slicer(exp,'end first permeability','end fast slip')

    df=slexp
    idxs=indexes

    print('total slip = '+str(df['slip'].iloc[-1]-df['slip'].iloc[0]))
    
    fig2,ax=plt.subplots(figsize=(14,9))
    ec=ax.scatter(df['effstress'], df['shear1'], c=df['Pf'], s=5, cmap=colormap)

    cbar = plt.colorbar(ec)
    cbar.set_label('Pf [MPa]')
    ec.set_clim(2,12)

    ax.set_xlim(0,16)

    ax.plot([0,16],[0,0.35*16],'--k',label='Slip tendency')
    ax.plot([0,16],[0,0.53*16],'--k',label='Byerlee')

    ax.set_title(expname)
    ax.set_xlabel('Effective normal stress [MPa]')
    ax.set_ylabel('Shear stress [MPa]')


    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['effstress'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('index out of bound')

    savefigures(expname+'_fig2',plots_path,fig2)
    # plt.close(fig2)

def figure2bis(exp,indexes,expname,plots_path):
    # %% figure 2 bis normal stress vs shear stress
    # % only injection section

    slexp,slindexes=slicer(exp,'end first permeability','end fast slip')

    df=slexp
    idxs=indexes
    
    fig2bis,ax=plt.subplots(figsize=(14,9))
    ec=ax.scatter(df['effstress'], df['shear1'], c=df['vel'], s=5, cmap=colormap,norm=mpl.colors.LogNorm())

    cbar = plt.colorbar(ec)
    cbar.set_label('Velocity [m/s]')
    ec.set_clim(1e-7,1)

    ax.set_xlim(0,16)

    ax.plot([0,16],[0,0.35*16],'--k',label='Slip tendency')
    ax.plot([0,16],[0,0.53*16],'--k',label='Byerlee')

    ax.set_title(expname)
    ax.set_xlabel('Effective normal stress [MPa]')
    ax.set_ylabel('Shear stress [MPa]')

    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['effstress'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('index out of bound')

    savefigures(expname+'_fig2bis',plots_path,fig2bis)
    # plt.close(fig2bis)

def figure3(exp,indexes,expname,plots_path):
    # %% figure 3 time vs slip (1 non log, 2 log)
    # % only injection section

    slexp,slindexes=slicer(exp,'end first permeability','end fast slip')

    df=slexp
    idxs=indexes

    s0=df['Slip_Enc_1'].iloc[0]
    df['Slip_Enc_1'].sub(s0)

    # df['Slip_Enc_1_filt'] = ssi.savgol_filter(df['Slip_Enc_1'], 31, 3)
    # diff = df.diff()
    # coords = [c for c in df.columns if 'Slip_Enc_1_filt' in c]
    # df['vel_filt']=np.linalg.norm(diff[coords], axis=1)/diff['Time']

    fig3,ax=plt.subplots(figsize=(14,9))
    ec=ax.scatter(df['Time_min'], df['Slip_Enc_1'], c=df['Pf'], s=5, cmap=colormap)

    cbar = plt.colorbar(ec)
    cbar.set_label('Pf [MPa]')
    ec.set_clim(2,12)

    ax.set_title(expname)
    ax.set_xlabel('Time [min]')
    ax.set_ylabel('slip [m]')
    ax.set_yscale('log')
    ax.set_ylim(1e-7,1)

    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['effstress'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('index out of bound')

    savefigures(expname+'_fig3',plots_path,fig3)
    # plt.close(fig3)

def figure4(exp,indexes,expname,plots_path):
    # %% figure 4 time vs layer thickness change
    # % only injection section

    slexp,slindexes=slicer(exp,'end first permeability','end fast slip')

    df=slexp
    idxs=indexes
    
    try:
        th0=df['thickness_high'].iloc[0]
        df['delta_thickness']=df['thickness_high'].sub(th0).replace()
    except:
        print('no thickness_high')

    try:
        th0=df['LVDT_high'].iloc[0]
        df['delta_thickness']=df['LVDT_high'].sub(th0).replace()
    except:
        print('no LVDT_high')


    fig4,ax=plt.subplots(figsize=(14,9))
    ec=ax.scatter(df['Time_min'], df['delta_thickness'], c=df['Pf'], s=5, cmap=colormap)

    cbar = plt.colorbar(ec)
    cbar.set_label('Pf [MPa]')
    ec.set_clim(2,12)

    ax.set_title(expname)
    ax.set_xlabel('Time [min]')
    ax.set_ylabel('$\Delta$ Layer thickness [mm]')
    ax.set_ylim(lcmin,lcmax)

    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['effstress'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('index out of bound')

    savefigures(expname+'_fig4',plots_path,fig4)
    # plt.close(fig4)

def figure5(exp,indexes,expname,plots_path):
    # %% figure 5 Time vs slip rate
    # % only injection section

    slexp,slindexes=slicer(exp,'end first permeability','end fast slip')

    df=slexp
    idxs=indexes
    
    # df.loc[df['vel'] <= 1e-6, 'vel'] = 1e-10
    df['vel_filt'] = ssi.savgol_filter(df['vel'], 31, 3)

    fig5,ax=plt.subplots(figsize=(14,9))
    ec=ax.scatter(df['Time_min'], df['vel_filt'], c=df['Pf'], s=5, cmap=colormap)

    cbar = plt.colorbar(ec)
    cbar.set_label('Pf [MPa]')
    ec.set_clim(2,12)

    ax.set_title(expname)
    ax.set_xlabel('Time [min]')
    ax.set_ylabel('Slip Rate [m/s]')
    ax.set_yscale('log')
    ax.set_ylim(1e-7,1)

    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['Time_min'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('tertiary creep was earlier')

    savefigures(expname+'_fig5',plots_path,fig5)
    # plt.close(fig5)

def figure6(exp,indexes,event_start,event_end,expname,plots_path):

    # %% figure 6 dinamico
    slexp,slindexes=slicer(exp,event_start,event_end)

    df=slexp
    idxs=indexes

    s0=df['Slip_Enc_1'].iloc[0]
    df['Slip_Enc_1'].sub(s0)
    t0=df['Time_min'].iloc[0]
    df['Time_min'].sub(t0)

    fig6,ax=plt.subplots(figsize=(14,9))
    ax.plot(df['Time_min'],df['shear1'],'-k',label='Shear stress')
    ax.plot(df['Time_min'],df['Pf'],'-b',label='Downstream pore pressure')
    # ax.plot(df['Time_min'],df['InternalPressure'],'-c',label='Upstream pore pressure')
    ax.plot(df['Time_min'],df['PumpPressure'],'-c',label='Upstream pore pressure')


    ax.set_title(expname)
    ax.set_xlabel('Time [min]')
    ax.set_ylabel('Shear stress, pore pressure [MPa]')
    ax.set_ylim([-0.2,7])

    ax1=ax.twinx()

    ax1.plot(df['Time_min'],df['vel'],'-',color='orange',label='Slip rate')
    ax1.set_ylim([1e-7,1])
    ax1.set_ylabel('Slip Rate [m/s]')
    ax1.set_yscale('log')

    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['Time_min'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('tertiary creep was earlier')
        
    try:
        i=idxs.loc[idxs['event_name']=='start last injection step',expname].iloc[0]
        ax.axvline(df['Time_min'][i],linestyle='dotted',color='b',label='Pf')
    except:
        print('Pf was earlier')

    # ax.legend('upper right')

    savefigures(expname+'_fig6_'+event_start+'_to_'+event_end,plots_path,fig6)
    # plt.close(fig6)

def figure6as(exp,indexes,event_start,event_end,expname,plots_path):

    # %% figure 6 as dinamico
    slexp,slindexes=slicer(exp,event_start,event_end)

    df=slexp
    idxs=indexes
    
    s0=df['Slip_Enc_1'].iloc[0]
    df['Slip_Enc_1']=df['Slip_Enc_1'].sub(s0)
    t0=df['Time_min'].iloc[0]
    df['Time_min']=df['Time_min'].sub(t0)


    fig6as,ax=plt.subplots(figsize=(14,9))
    ax.plot(df['Time_min'],df['shear1'],'-k',label='Shear stress')
    ax.plot(df['Time_min'],df['Pf'],'-b',label='Downstream pore pressure')
    # ax.plot(df['Time_min'],df['InternalPressure'],'-c',label='Upstream pore pressure')
    ax.plot(df['Time_min'],df['PumpPressure'],'-c',label='Upstream pore pressure')

    ax.set_title(expname)
    ax.set_xlabel('Time [min]')
    ax.set_ylabel('Shear stress, pore pressure [MPa]')    
    ax.set_ylim([-0.2,7])

    ax1=ax.twinx()

    ax1.plot(df['Time_min'],df['Slip_Enc_1'],'-g',label='Slip')
    ax1.set_ylim([1e-7,1])
    ax1.set_ylabel('Slip [m]')
    ax1.set_yscale('log')

    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['Time_min'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('tertiary creep was earlier')
        
    try:
        i=idxs.loc[idxs['event_name']=='start last injection step',expname].iloc[0]
        ax.axvline(df['Time_min'][i],linestyle='dotted',color='b',label='Pf')
    except:
        print('Pf was earlier')

    # ax.legend('upper right')

    savefigures(expname+'_fig6as_'+event_start+'_to_'+event_end,plots_path,fig6as)
    # plt.close(fig6)

def figure6as2(exp,indexes,event_start,event_end,expname,plots_path):

    # %% figure 6 as2 dinamico
    slexp,slindexes=slicer(exp,event_start,event_end)

    df=slexp
    idxs=indexes

    s0=df['Slip_Enc_1'].iloc[0]
    df['Slip_Enc_1'].sub(s0)
    
    t0=df['Time_min'].iloc[0]
    df['Time_min'].sub(t0)

    try:
        th0=df['thickness_high'].iloc[0]
        df['delta_thickness']=df['thickness_high'].sub(th0).replace()
    except:
        print('no thickness_high')

    try:
        th0=df['LVDT_high'].iloc[0]
        df['delta_thickness']=df['LVDT_high'].sub(th0).replace()
    except:
        print('no LVDT_high')

    fig6as2,ax=plt.subplots(figsize=(14,9))
    ax.plot(df['Time_min'],df['shear1'],'-k',label='Shear stress')
    ax.plot(df['Time_min'],df['Pf'],'-b',label='Downstream pore pressure')
    # ax.plot(df['Time_min'],df['InternalPressure'],'-c',label='Upstream pore pressure')
    ax.plot(df['Time_min'],df['PumpPressure'],'-c',label='Upstream pore pressure')


    ax.set_title(expname)
    ax.set_xlabel('Time [min]')
    ax.set_ylabel('Shear stress, pore pressure [MPa]')
    ax.set_ylim([-0.2,7])


    ax1=ax.twinx()

    ax1.plot(df['Time_min'],df['delta_thickness'],'-',color='lightgrey')
    ax1.set_ylim([-0.07,0.07])
    ax1.set_ylabel('$\Delta$ Layer thickness [mm]')

    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['Time_min'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('tertiary creep was earlier')
        
    try:
        i=idxs.loc[idxs['event_name']=='start last injection step',expname].iloc[0]
        ax.axvline(df['Time_min'][i],linestyle='dotted',color='b',label='Pf')
    except:
        print('Pf was earlier')

    # ax.legend('upper right')

    savefigures(expname+'_fig6as2_'+event_start+'_to_'+event_end,plots_path,fig6as2)
    # plt.close(fig6)

def figure6bis(exp,indexes,event_start,event_end,expname,plots_path):

    # %% figure 6bis dinamico

    slexp,slindexes=slicer(exp,event_start,event_end)

    df=slexp
    idxs=indexes
    
    fig6bis,ax=plt.subplots(figsize=(14,9))
    ax.plot(df['Time_min'],df['Mu_eff'],'-b',label='Effective friction')
    # ax.plot(df['Time_min'],df['Mu_app'],'--b',label='Apparent friction')

    ax.set_title(expname)
    ax.set_xlabel('Time [min]')
    ax.set_ylabel('Effective friction')
    ax.set_ylim([0,0.8])

    ax1=ax.twinx()

    ax1.plot(df['Time_min'],df['vel'],'-k',label='Slip rate')
    ax1.set_ylim([1e-7,1])
    ax1.set_ylabel('Slip Rate [m/s]')
    ax1.set_yscale('log')

    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['Time_min'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('tertiary creep was earlier')

    try:
        i=idxs.loc[idxs['event_name']=='start last injection step',expname].iloc[0]
        ax.axvline(df['Time_min'][i],linestyle='dotted',color='b',label='Pf')
    except:
        print('Pf was earlier')
        
    # ax.legend('upper right')

    savefigures(expname+'_fig6bis_'+event_start+'_to_'+event_end,plots_path,fig6bis)

    # plt.close(fig6bis)

def figure6tris(exp,indexes,event_start,event_end,expname,plots_path):

    # %% figure 6tris

    slexp,slindexes=slicer(exp,event_start,event_end)

    df=slexp
    idxs=indexes
        
    fig6tris,ax=plt.subplots(figsize=(14,9))
    ax.plot(df['slip'],df['Mu_eff'],'-b',label='Effective friction')
    # ax.plot(df['Time_min'],df['Mu_app'],'--b',label='Apparent friction')

    ax.set_title(expname)
    ax.set_xlabel('Slip [m]')
    ax.set_ylabel('Effective friction')
    ax.set_ylim([0,0.8])

    ax1=ax.twinx()

    ax1.plot(df['slip'],df['vel'],'-k',label='Slip rate')
    ax1.set_ylim([1e-7,1])
    ax1.set_ylabel('Slip Rate [m/s]')
    ax1.set_yscale('log')

    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['slip'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('tertiary creep was earlier')
        
    try:
        i=idxs.loc[idxs['event_name']=='start last injection step',expname].iloc[0]
        ax.axvline(df['slip'][i],linestyle='dotted',color='b',label='Pf')
    except:
        print('Pf was earlier')
            
    # ax.legend('upper right')

    savefigures(expname+'_fig6tris_'+event_start+'_to_'+event_end,plots_path,fig6tris)

    # plt.close(fig6tris)

def figure6quad(exp,indexes,event_start,event_end,expname,plots_path):

    # %% figure 6quad
    slexp,slindexes=slicer(exp,event_start,event_end)

    df=slexp
    idxs=indexes    
    fig6quad,ax=plt.subplots(figsize=(14,9))
    fig6quad.subplots_adjust(right=0.75)

    ax.plot(df['slip'],df['Mu_eff'],'-b',label='Effective friction')
    # ax.plot(df['Time_min'],df['Mu_app'],'--b',label='Apparent friction')

    ax.set_title(expname)
    ax.set_xlabel('Slip [m]')
    ax.set_ylabel('Effective friction')
    ax.set_ylim([0,0.8])

    ax1=ax.twinx()

    ax1.plot(df['slip'],df['vel'],'-k',label='Slip rate')
    ax1.set_ylim([1e-7,1])
    ax1.set_ylabel('Slip Rate [m/s]')
    ax1.set_yscale('log')

    ax2=ax.twinx()
    ax2.set_ylim([0,50])
    ax2.set_ylabel('Temperature [°C]')
    ax2.spines['right'].set_position(("axes", 1.2))
    try:
        ax2.plot(df['slip'],df['InternalTemperature'],'-r',label='Temperature')
    except:
        print('no temperature')

    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['Slip_Enc_1'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('tertiary creep was earlier')
        
    try:
        i=idxs.loc[idxs['event_name']=='start last injection step',expname].iloc[0]
        ax.axvline(df['Slip_Enc_1'][i],linestyle='dotted',color='b',label='Pf')
    except:
        print('Pf was earlier')

    # ax.legend('upper right')

    savefigures(expname+'_fig6quad_'+event_start+'_to_'+event_end,plots_path,fig6quad)

    # plt.close(fig6quad)

def figure6cinq(exp,indexes,event_start,event_end,expname,plots_path):

    # %% figure 6cinq
    slexp,slindexes=slicer(exp,event_start,event_end)

    df=slexp
    idxs=indexes    

    try:
        th0=df['thickness_high'].iloc[0]
        df['delta_thickness']=df['thickness_high'].sub(th0).replace()
    except:
        print('no thickness_high')

    try:
        th0=df['LVDT_high'].iloc[0]
        df['delta_thickness']=df['LVDT_high'].sub(th0).replace()
    except:
        print('no LVDT_high')

    df['Mu_eff_filt'] = ssi.savgol_filter(df['Mu_eff'], 15, 3)

    fig6cinq,ax=plt.subplots(figsize=(14,9))
    fig6cinq.subplots_adjust(right=0.75)

    ax.plot(df['slip'],df['Mu_eff_filt'],'-b',label='Effective friction')
    # ax.plot(df['Time_min'],df['Mu_app'],'--b',label='Apparent friction')

    ax.set_title(expname)
    ax.set_xlabel('Slip [m]')
    ax.set_ylabel('Effective friction')
    ax.set_ylim([0,0.8])

    ax1=ax.twinx()

    ax1.plot(df['slip'],df['vel'],'-k',label='Slip rate')
    ax1.set_ylim([1e-7,1])
    ax1.set_ylabel('Slip Rate [m/s]')
    ax1.set_yscale('log')

    ax2=ax.twinx()

    ax2.plot(df['slip'],df['delta_thickness'],'-r',label='$\Delta$ Layer thickness [mm]')
    ax2.set_ylim([-0.07,0.27])
    ax2.set_ylabel('$\Delta$ Layer thickness [mm]')
    ax2.spines['right'].set_position(("axes", 1.2))
    
    # ax.legend('upper right')

    savefigures(expname+'_fig6cinq_'+event_start+'_to_'+event_end,plots_path,fig6cinq)

    # plt.close(fig6cinq)

def figure7(exp,indexes,event_start,event_end,expname,plots_path):

    # %% figure 7 dinamico more complicated (stile hikurangi)
    slexp,slindexes=slicer(exp,event_start,event_end)

    df=slexp
    idxs=indexes

    s0=df['Slip_Enc_1'].iloc[0]
    df['Slip_Enc_1'].sub(s0)
        
    t0=df['Time_min'].iloc[0]
    df['Time_min'].sub(t0)

    try:
        th0=df['thickness_high'].iloc[0]
        df['delta_thickness']=df['thickness_high'].sub(th0).replace()
    except:
        print('no thickness_high')

    try:
        th0=df['LVDT_high'].iloc[0]
        df['delta_thickness']=df['LVDT_high'].sub(th0).replace()
    except:
        print('no LVDT_high')



    fig7,ax=plt.subplots(figsize=(14,9))
    ax.plot(df['Time_min'],df['shear1'],'-',color='black',label='Shear stress')

    #shear stress
    ax.set_title(expname)
    ax.set_xlabel('Time [min]')
    ax.set_ylabel('Shear stress [MPa]')
    # ax.set_ylim([0,0.8])

    #slip rate
    ax1=ax.twinx()

    ax1.plot(df['Time_min'],df['vel'],'-',color='orange',label='Slip rate')
    ax1.set_ylim([1e-7,1])
    ax1.set_ylabel('Slip Rate [m/s]')
    ax1.set_yscale('log')

    #pore pressures
    ax2=ax.twinx()
    try:
        if expname=='brava':
            ax2.plot(df['Time_min'],df['InternalPressure'],'-',color='c',label='Upstream pore pressure')
        else:
            ax2.plot(df['Time_min'],df['PumpPressure'],'-',color='c',label='Upstream pore pressure')
    except:
        print('nothing')

    ax2.plot(df['Time_min'],df['Pf'],'-',color='b',label='Downstream pore pressure')
    # ax2.set_ylim([1e-7,1])
    ax2.set_ylabel('Pore pressure [MPa]')
    ax2.spines["right"].set_position(("axes",-0.2))

    #dilatancy
    ax3=ax.twinx()
    ax3.plot(df['Time_min'],df['delta_thickness'],'-',color='lightgrey',label='Downstream pore pressure')

    # ax2.set_ylim([1e-7,1])
    ax3.set_ylabel('$\Delta$ Thickness [mm]')
    ax3.spines["right"].set_position(("axes",1.1))


    try:
        i=idxs.loc[idxs['event_name']=='onset of tertiary creep',expname].iloc[0]
        ax.axvline(df['Time_min'][i],linestyle='dotted',color='r',label='t. c.')
    except:
        print('tertiary creep was earlier')

    try:
        i=idxs.loc[idxs['event_name']=='start last injection step',expname].iloc[0]
        ax.axvline(df['Time_min'][i],linestyle='dotted',color='b',label='Pf')
    except:
        print('Pf was earlier')

    #LEGEND
    legend_elements = [Line2D([0], [0], color='k', lw=1, label='Shear strength'),
                        Line2D([0], [0], color='c', linestyle='-', label='Upstream Pf'),
                        Line2D([0], [0], color='b', linestyle='-', label='Downstream Pf'),
                        Line2D([0], [0], color='orange', linestyle='-', label='Slip velocity'),
                        Line2D([0], [0], color='lightgrey', linestyle='-', label='$\Delta$ thickness'),
                        ]
    fig7.legend(handles=legend_elements,bbox_to_anchor=(0, 0.02, 1., .102), loc='lower left',
                ncol=6, mode="expand", borderaxespad=0.1)

    fig7.subplots_adjust(right=0.85,wspace=0.05,hspace=0.1,left=0.15,top=0.95,bottom=0.15)

    ax.legend('upper right')

    savefigures(expname+'_fig7_'+event_start+'_to_'+event_end,plots_path,fig7)
    # plt.close(fig7)

### Calling functions above depending on experiment and related metadata

In [8]:
expname="s1949"

if expname=="s1949":
    print("s1949")
    mat=sio.loadmat(shiva_red_unix_path+'s1949.mRED.mat')
    exp=mat2df(mat)

    ismat=True
    machine='shiva'
    material='gouge'

    exp=mapper(exp)
    
elif expname=="s1984_r1":
    print("s1984_r1")
    mat=sio.loadmat(shiva_red_unix_path+'s1984.mRED.mat')
    exp=mat2df(mat)

    ismat=True
    machine='shiva'
    material='gouge'

    exp=mapper(exp)
    
elif expname=="s1984_r2":
    print("s1984_r2") 
    mat=sio.loadmat(shiva_red_unix_path+'s1984.mRED.mat')
    exp=mat2df(mat)

    ismat=True
    machine='shiva'
    material='gouge'

    exp=mapper(exp)

elif expname=="s1997":
    print("s1997") 
    mat=sio.loadmat(shiva_red_unix_path+'s1997.mRED.mat')
    exp=mat2df(mat)

    ismat=True
    machine='shiva'
    material='rock'
    
    exp=mapper(exp)

# elif expname=='brava':
#     print('brava')
#     exp=pd.read_csv('../data/dsv/Copia di MC_Injection.csv',sep='\t')

#     ismat=False
#     machine='brava'
#     material='gouge'

#     exp=mapper(exp)

else:
    print("ciao")

s1949


### Saving the called experiment to a file 
To avoid repeating operations above and do comparison with models

In [19]:
if os.path.isfile(shiva_red_jupyter_path+expname+'.mat'):
    print(expname+'.mat already exists')
else:
    print(expname+'.mat is being made')
    # exp.size
    exp_dict= {name: col.values for name, col in exp.items()}
    # exp_dict
    sio.savemat(shiva_red_jupyter_path+expname+'.mat', exp_dict, do_compression=True)  

s1949.mat already exists


### Generating figures

In [20]:
# for figure 3 of the paper
figure6cinq(exp,indexes,'start second-to-last injection step','end fast slip',expname,plots_path)

# for supplementary figure 3 of the paper
figure6as(exp,indexes,'start first injection step','end fast slip',expname,plots_path)

C:\Users\stear\AppData\Local\Temp\ipykernel_18016\1306787961.py:556: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df['delta_thickness']=df['thickness_high'].sub(th0).replace()
C:\Users\stear\AppData\Local\Temp\ipykernel_18016\1306787961.py:556: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['delta_thickness']=df['thickness_high'].sub(th0).replace()
C:\Users\stear\AppData\Local\Temp\ipykernel_18016\1306787961.py:562: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df['delta_thickness']=df[

In [21]:
figure6(exp,indexes,'start preshear','end last permeability',expname,plots_path)
figure6as(exp,indexes,'start preshear','end last permeability',expname,plots_path)
figure6as2(exp,indexes,'start preshear','end last permeability',expname,plots_path)

#mohr plot (preshear to end of fast slip)
figure_mohr(exp,indexes,expname,plots_path)

#resume plot (preshear to last permeability)
figure1(exp,indexes,expname,plots_path)

# reactivation plots (start second to last step to end of fast slip)

figure6(exp,indexes,'start second-to-last injection step','end fast slip',expname,plots_path)
figure6as(exp,indexes,'start second-to-last injection step','end fast slip',expname,plots_path)
figure6as2(exp,indexes,'start second-to-last injection step','end fast slip',expname,plots_path)

# focus on dynamic stage
figure6bis(exp,indexes,'start second-to-last injection step','end fast slip',expname,plots_path)
figure6tris(exp,indexes,'start second-to-last injection step','end fast slip',expname,plots_path)
figure6quad(exp,indexes,'start second-to-last injection step','end fast slip',expname,plots_path)

# figure7(exp,indexes,'start second-to-last injection step','end fast slip',expname,plots_path)

# full reactivation stage
figure6(exp,indexes,'start first injection step','end fast slip',expname,plots_path)
figure6as(exp,indexes,'start first injection step','end fast slip',expname,plots_path)
figure6as2(exp,indexes,'start first injection step','end fast slip',expname,plots_path)

figure7(exp,indexes,'start first injection step','end fast slip',expname,plots_path)

C:\Users\stear\AppData\Local\Temp\ipykernel_18016\1306787961.py:306: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Slip_Enc_1']=df['Slip_Enc_1'].sub(s0)
C:\Users\stear\AppData\Local\Temp\ipykernel_18016\1306787961.py:308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time_min']=df['Time_min'].sub(t0)
C:\Users\stear\AppData\Local\Temp\ipykernel_18016\1306787961.py:362: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. E

### Plotting and extracting events
Just to play with plotted data. It prints results to ouput, it does not update any table.

In [22]:
# plot for finding events

fig,ax=plt.subplots()


ax.plot(exp['shear1'])
ax.plot(exp['Normal'])
ax.plot(exp['PumpPressure'])
ax.plot(exp['vel'])
ax.plot(exp['slip'])

try:
    i=indexes.loc[indexes['event_name']=='start last injection step',expname].iloc[0]
    ax.axvline([i],linestyle='dotted',color='b',label='Pf')
except:
    print('Pf was earlier')

x=fig.ginput(n=5, timeout=120, show_clicks=True)
print(x)

[(996303.0846774192, 5.319683478470912), (1246802.145967742, 5.455330052549247), (1539051.0508064516, 5.5909766266275795), (1862612.3383064512, 5.658799913666746), (2196611.086693548, 5.794446487745079)]


### Extracting slip and effective friction related to event

In [23]:
# print event data
print(expname)

for value in x:
    rn=round(value[0])
    print(str(rn))
    print('Mu_eff = '+str(exp['Mu_eff'][rn]))
    print('slip = '+str(exp['slip'][rn]))

s1949
996303
Mu_eff = 0.4526636690438647
slip = 0.009632123075906308
1246802
Mu_eff = 0.456143831927858
slip = 0.009632123075906308
1539051
Mu_eff = 0.45973682178374975
slip = 0.009632123075906308
1862612
Mu_eff = 0.46804744046698277
slip = 0.009632123075906308
2196611
Mu_eff = 0.4841507971819603
slip = 0.009632123075906308


### Slicing experiment data to proceed with permeability estimation

In [34]:
# different logic with respect to vajont
dt2={'exp':'str','event_id':'int','event_name':'str','start':'int','end':'int'}
perm_indexes=pd.read_excel(table_path+'experiment_stages.xlsx',sheet_name='indexes-permeability',usecols=[0,1,2,3,4],dtype=dt2)

perm_indexes=perm_indexes.rename(columns={"cycle": "event_id", "type": "event_name"})
perm_indexes.head()

# slice the file for permeability stages
def slice_and_save(path,name,indexes):
    for i in range(max(indexes[indexes['exp']==name]['event_id'].unique())):
        i=i+1

        a=indexes[(indexes['exp']==name) & (indexes['event_id']==i)]['start'].reset_index(drop=True)[0]
        b=indexes[(indexes['exp']==name) & (indexes['event_id']==i)]['end'].reset_index(drop=True)[0]
        # print(str(a)+' '+str(b))
        if a==0 & b==0:
            print('nothing')
        else:
            slicedf=exp.iloc[a:b]
            slicedf.to_csv(path+name+'_'+str(i)+'.dat',columns=['Pf','PumpPressure','Time','PumpVolume','Pc'],index=False)

In [35]:
slice_and_save(out_perm_data_path,expname,perm_indexes)